In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import Literal
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

In [3]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [4]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [5]:
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

In [6]:
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [7]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

In [8]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [9]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}},
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='e1d5cf60-5793-437b-8540-0f6d78b4dbe1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bN4EmaZi0wa077xVbRP76PFt', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dd2fc0a4-4b73-4d3f-b269-b8c1570b0734-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_bN4EmaZi0wa077xVbRP76PFt', 'type': 'tool_call'}], usage_metadata={

In [10]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city then?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
)

{'messages': [HumanMessage(content='How is the weather in munich?', additional_kwargs={}, response_metadata={}, id='e1d5cf60-5793-437b-8540-0f6d78b4dbe1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bN4EmaZi0wa077xVbRP76PFt', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dd2fc0a4-4b73-4d3f-b269-b8c1570b0734-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_bN4EmaZi0wa077xVbRP76PFt', 'type': 'tool_call'}], usage_metadata={

### Getting production ready - async and streaming

In [11]:
model = ChatOpenAI(model="gpt-4o-mini", streaming=True).bind_tools(tools)

In [12]:
async def call_model(state: MessagesState):
    messages = state["messages"]
    response = await model.ainvoke(messages)
    return {"messages": [response]}

In [13]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {"tools": "tools", END: END},
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=MemorySaver())

In [14]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
config = {"configurable": {"thread_id": 2}}

In [15]:
await graph.ainvoke(input=inputs, config=config)

{'messages': [HumanMessage(content='How is the weather in Munich?', additional_kwargs={}, response_metadata={}, id='596d16a6-7694-4243-ae41-88b683bbb79c'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jOIflsG21EcvzSDxSvJsyTKw', 'function': {'arguments': '{"location":"Munich"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-52339c26-7508-426d-a39b-16200f94859d-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Munich'}, 'id': 'call_jOIflsG21EcvzSDxSvJsyTKw', 'type': 'tool_call'}]),
  ToolMessage(content="It's 15 degrees Celsius and cloudy.", name='get_weather', id='5254352d-0aa4-4215-ac7d-d36d4865b579', tool_call_id='call_jOIflsG21EcvzSDxSvJsyTKw'),
  AIMessage(content='The weather in Munich is currently 15 degrees Celsius and cloudy.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 

In [16]:
inputs = {"messages": [HumanMessage(content="How is the weather in Munich?")]}
async for output in graph.astream(inputs, stream_mode="updates", config=config):
    # stream_mode="updates" yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value["messages"][-1].pretty_print())

Output from node 'agent':
---
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_3JjjzsuNcmrh0Z3y1d4qosUl)
 Call ID: call_3JjjzsuNcmrh0Z3y1d4qosUl
  Args:
    location: Munich
None
Output from node 'tools':
---
================================= Tool Message =================================
Name: get_weather

It's 15 degrees Celsius and cloudy.
None
Output from node 'agent':
---
================================== Ai Message ==================================

The weather in Munich is currently 15 degrees Celsius and cloudy.
None


In [17]:
from langchain_core.messages import AIMessageChunk, HumanMessage

inputs = [HumanMessage(content="How is the weather in Munich?")]
gathered = None

async for msg, metadata in graph.astream(
    {"messages": inputs}, stream_mode="messages", config=config
):
    if msg.content and not isinstance(msg, HumanMessage):
        # Print each token as it streams in
        print(msg.content, end="|", flush=True)

    # Handle the AI message chunks for proper assembly
    if isinstance(msg, AIMessageChunk):
        if gathered is None:
            gathered = msg
        else:
            gathered = gathered + msg

It's 15 degrees Celsius and cloudy.|The| weather| in| Munich| is| currently| |15| degrees| Celsius| and| cloudy|.|

In [18]:
print(gathered.content)

The weather in Munich is currently 15 degrees Celsius and cloudy.
